In [1]:
from llama_index.llms.gemini import Gemini
from dotenv import load_dotenv

c:\Users\divyakirant\Documents\llamaIndex\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

In [3]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,Settings
from llama_index.embeddings.gemini import GeminiEmbedding


# Configure Gemini models
gemini_llm = Gemini(model="models/gemini-1.5-pro")
gemini_embedding = GeminiEmbedding(model="models/embedding-001")

# Tell LlamaIndex to use Gemini everywhere
Settings.llm = gemini_llm
Settings.embed_model = gemini_embedding


C:\Users\divyakirant\AppData\Local\Temp\ipykernel_20664\2549428705.py:6: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  gemini_llm = Gemini(model="models/gemini-1.5-pro")
C:\Users\divyakirant\AppData\Local\Temp\ipykernel_20664\2549428705.py:7: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  gemini_embedding = GeminiEmbedding(model="models/embedding-001")


In [4]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex

In [5]:
from llama_index.core.memory import ChatSummaryMemoryBuffer

In [6]:
memory=ChatSummaryMemoryBuffer.from_defaults(token_limit=2000)

In [7]:
reader = SimpleDirectoryReader(input_files=['./data/ragintro.pdf'])
documents = reader.load_data()

In [8]:
parser = SentenceSplitter(chunk_size=100, chunk_overlap=20)
all_documents = parser.get_nodes_from_documents(documents)

In [9]:
# store the documents in a vector store index
index = VectorStoreIndex.from_documents(documents=all_documents)

print(f"Vectors Store: {index}")
print(f"Vectors Store ID: {index.index_id}")

Vectors Store: <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x0000021EA490AF90>
Vectors Store ID: 47bc7442-3490-4f9d-8257-1e1534649987


In [10]:
chat_engine=index.as_chat_engine(
    chat_mode="context",
    llm=gemini_llm,
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about an pdf loaded in this context.If the question is not relevant to the context and in the loaded documents, just say I don't know."
    )
)

In [11]:
response = chat_engine.chat("various strategies of chunking from the pdf")
print(response)

Okay, I'm ready to chat and answer your questions about the provided PDF excerpts.  Specifically, I can tell you about different chunking strategies, particularly regarding file format-based chunking.  For example, I know that it's recommended to keep HTML elements like tables and images intact during chunking, and similar considerations should be applied to PDFs.  I also know about the key RAG stages: Indexing, Retrieval, and Augmentation.  I even have some page number references for this information.

What can I tell you about chunking or RAG?



In [12]:
# print the chat history
print("\nChat History:")
print(memory.get())


Chat History:
[ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='various strategies of chunking from the pdf')]), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="Okay, I'm ready to chat and answer your questions about the provided PDF excerpts.  Specifically, I can tell you about different chunking strategies, particularly regarding file format-based chunking.  For example, I know that it's recommended to keep HTML elements like tables and images intact during chunking, and similar considerations should be applied to PDFs.  I also know about the key RAG stages: Indexing, Retrieval, and Augmentation.  I even have some page number references for this information.\n\nWhat can I tell you about chunking or RAG?\n")])]


In [13]:
response = chat_engine.chat("can you explain more ")
print(response)

Sure! Based on the context you provided, I can elaborate on a few points related to chunking and Retrieval Augmented Generation (RAG):

* **Chunking in the context of PDFs:**  While the provided text doesn't explicitly detail *specific* chunking strategies for PDFs, it implies the importance of thoughtful chunking, similar to HTML.  This suggests that preserving the logical structure of the PDF is crucial.  Think about how a PDF might be organized: sections, subsections, figures, tables, etc.  Ideal chunking would likely keep these elements together rather than splitting them arbitrarily.  For example, a whole table should be in one chunk, not split across multiple chunks.  This is important for maintaining context and ensuring that the retrieved information is coherent.

* **Connection to RAG:**  Chunking is the first step in the "Indexing" stage of RAG.  The PDF is broken down into smaller, manageable pieces (chunks) that can then be indexed by a search engine.  This allows the syste